<a href="https://colab.research.google.com/github/jlopez12/Analisis_prediccion_de_ventas-Part_1/blob/development/Analisis_prediccion_ventas_Part-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Conectar a Drive y leer documento

In [ ]:
# Conectar Colab con drive.
from google.colab import drive
drive.mount('/content/drive')

: 

In [ ]:
# Importa libreria.
import pandas as pd

In [ ]:
# Establecer conexion al documento
path = "/content/drive/MyDrive/Colab Notebooks/DB/retail_sales_dataset.csv"
df = pd.read_csv(path)

#Primera vista al Data Frame

In [ ]:
# Se revisan las columnas y tipos de datos.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    1000 non-null   int64 
 1   Date              1000 non-null   object
 2   Customer ID       1000 non-null   object
 3   Gender            1000 non-null   object
 4   Age               1000 non-null   int64 
 5   Product Category  1000 non-null   object
 6   Quantity          1000 non-null   int64 
 7   Price per Unit    1000 non-null   int64 
 8   Total Amount      1000 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 70.4+ KB


#Limpieza de columnas

In [ ]:
# Pasa todas las columnas a minusculas.
df.columns = df.columns.str.lower()

In [ ]:
# Diccionario de caracteres que quiero reemplazar en los nombres de columnas.
# Aunque no se visualizan tildes, parentesis o ° se mantiene en el diccionario.
reemplazar = {
    "á": "a", "é": "e", "í": "i", "ó": "o", "ú": "u",
    " ":"_","(":"",")":"","°":"",".":""
    }

In [ ]:
# Se recorre el diccionario de caracteres a reemplazar y se aplica en las columnas
for actual, nuevo in reemplazar.items():
  df.columns = df.columns.str.replace(actual,nuevo)

In [ ]:
# Se verifican los cambios.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_id    1000 non-null   int64 
 1   date              1000 non-null   object
 2   customer_id       1000 non-null   object
 3   gender            1000 non-null   object
 4   age               1000 non-null   int64 
 5   product_category  1000 non-null   object
 6   quantity          1000 non-null   int64 
 7   price_per_unit    1000 non-null   int64 
 8   total_amount      1000 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 70.4+ KB


* Transformación de Datos

Crea nuevas columnas: Basándonos en los datos existentes, crea nuevas columnas que sean útiles para el análisis. Por ejemplo, calcula el ingreso total por venta y normaliza las ventas.

In [ ]:
# Crea columna Total Venta para verificar el Total amount
df["total_venta"] = df["quantity"] * df["price_per_unit"]

Clasifica los datos: Crea una columna que clasifique las ventas en categorías significativas (e.g., ‘Alta’, ‘Media’, ‘Baja’).

In [ ]:
# Funcion para clasifivar venta segun Total Venta.
def venta_tipo(total):
  if total <= 200:
    return 'Baja'
  elif total <= 300:
    return 'Media'
  else:
    return 'Alta'

In [ ]:
# Mediante apply se verifica la clasificacion de la venta.
df["Clasificacion"] = df["Total Venta"].apply(venta_tipo)

In [ ]:
# Verifica la columna creada.
df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Total Venta,Clasificacion
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,150,Baja
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,1000,Alta
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,30,Baja
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,500,Alta
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,100,Baja


Ejemplo de uso avanzado: Calcula la desviación de cada venta respecto a la media de su grupo.

In [ ]:
# Funcion para calcular la desviacion.
def calcular_std(grupo):
    # Obtiene el promedio del total de ventas
    prom = grupo['Total Venta'].mean().round(2)
    # Crea columna donde el valor sea el total de venta menos el promedio para mostrar cuanto se desvia cada venta el promedio.
    grupo['Desviacion_por_Total'] = grupo['Total Venta'] - prom
    return grupo

# Agrupa por categoria de productos y aplicar el calculo.
df = df.groupby('Product Category').apply(calcular_std)


<ipython-input-75-2452689732>:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Product Category').apply(calcular_std)


In [ ]:
# Verifica la nueva columna
df.head(20)

Transaction ID        Date Customer ID  Gender  Age  \
Product Category                                                           
Beauty           0                1  2023-11-24     CUST001    Male   34   
                 4                5  2023-05-06     CUST005    Male   30   
                 5                6  2023-04-25     CUST006  Female   45   
                 11              12  2023-10-30     CUST012    Male   35   
                 20              21  2023-01-14     CUST021  Female   50   
                 24              25  2023-12-26     CUST025  Female   64   
                 26              27  2023-08-03     CUST027  Female   38   
                 27              28  2023-04-23     CUST028  Female   43   
                 29              30  2023-10-29     CUST030  Female   39   
                 31              32  2023-01-04     CUST032    Male   30   
                 34              35  2023-08-05     CUST035  Female   58   
                 35              36  2023-06-24     CUST036    Male   52   
                 36              37  2023-05-23     CUST037  Female   18   
                 37              38  2023-03-21     CUST038    Male   38   
                 39              40  2023-06-22     CUST040    Male   45   
                 46              47  2023-11-06     CUST047  Female   40   
                 49              50  2023-08-24     CUST050  Female   27   
                 50              51  2023-10-02     CUST051    Male   27   
                 51              52  2023-03-05     CUST052  Female   36   
                 54              55  2023-10-10     CUST055    Male   31   

                    Product Category  Quantity  Price per Unit  Total Amount  \
Product Category                                                               
Beauty           0            Beauty         3              50           150   
                 4            Beauty         2              50           100   
                 5            Beauty         1              30            30   
                 11           Beauty         3              25            75   
                 20           Beauty         1             500           500   
                 24           Beauty         1              50            50   
                 26           Beauty         2              25            50   
                 27           Beauty         1             500           500   
                 29           Beauty         3             300           900   
                 31           Beauty         3              30            90   
                 34           Beauty         3             300           900   
                 35           Beauty         3             300           900   
                 36           Beauty         3              25            75   
                 37           Beauty         4              50           200   
                 39           Beauty         1              50            50   
                 46           Beauty         3             500          1500   
                 49           Beauty         3              25            75   
                 50           Beauty         3              25            75   
                 51           Beauty         1             300           300   
                 54           Beauty         4              30           120   

                     Total Venta Clasificacion  Desviacion_por_Total  
Product Category                                                      
Beauty           0           150          Baja               -317.48  
                 4           100          Baja               -367.48  
                 5            30          Baja               -437.48  
                 11           75          Baja               -392.48  
                 20          500          Alta                 32.52  
                 24           50          Baja               -417.48  
                 26